In [ ]:
from TTVFit.ttvfit import *
from TTVFit.lightcurve import *
from TTVFit.getdata import *

In [ ]:
indir = 'wasp-107b/'
target_star = 'WASP-107'
target = 'WASP-107b'
period = 5.72148926
IDs = ['1185o8','1280o1','1201o8','1224o3','1185o9']
tess_path = 'TESS Timing data/WASP/WASP-107 b/tess_pytransit.txt'

In [ ]:
GD = GetData(target,indir,period,IDs,)
LC = LightCurve(target,indir,period)

In [ ]:
time = GD.getTime()
time

In [ ]:
t = GD.getT('HST.xlsx','tess.csv',tess_path,target_star)
t.loc[0,'Label'] = 'Ground'
t.loc[1,'Label'] = 'K2'
t

In [ ]:
t_re = GD.checkDuplicates(t)
t_re

In [ ]:
RE = True
if t_re is None:
    RE = False
if RE:
    data = t_re.copy()
else:
    data = t.copy()

In [ ]:
times0_pre_lin = 2456514.4115079916
period_pre_lin = 5.721487908373708
times0_pre = times0_pre_lin
period_pre = period_pre_lin
a = 0.005
times0_est = [times0_pre - a, times0_pre + a,'U']
b = 0.0001
# period_est = [period_pre-b, period_pre+b,'U']
period_est = [period_pre,b,'N']
# omega_est = [0,2*np.pi,'U']
omega_est = [2.93014,0.2,0,2*np.pi,'TN']
domega_est = [1e-6,1e-3,'LU']
# eccentricity_est = [1e-3,1e-1,'LU']
eccentricity_est = [0.1875,0.002,0,1,'TN']

precession_parameters = [
    eccentricity_est,
    omega_est,
    domega_est,
]

sampler_parameters = {
    'emcee':{
        'nwalkers':32,
        'nsteps':100000,
        'nburnin':15000,
    },
    'dynesty':{
        'nlive':1000,
        'dlogz':0.001,
    },
    'dynamic_dynesty':{
        'nlive_init':10000,
        'dlogz_init':0.0001,
    }
}

kind = 'Precession'
rej_sigma = 5
multiprocessing = True
# sampler = 'emcee'
# sampler = 'dynesty'
sampler = 'dynamic_dynesty'
parameters = [times0_est,period_est]
parameter_names = [r'$t_0$',r'$Period$']

parameters += precession_parameters
parameter_names += [r'$eccentricity$',r'$\omega[radian]$',r'$d\omega/dE[radian/epoch]$']

print('\n'.join(['%s = %s'%(k,v) for k,v in zip(parameter_names,parameters)]))

TTVFIT = TTVFit(data,period,parameters,parameter_names,kind,rej_sigma,multiprocessing=multiprocessing,sampler=sampler,sampler_parameters=sampler_parameters[sampler],lin_parameters=[times0_pre_lin,period_pre_lin])

In [ ]:
suffix = '_%ssigma'%rej_sigma
suffix += '_%s_%s_%s_%s'%(sampler,kind,target,parameters[-1])
if RE:
    suffix += '_reduced'
# suffix += ''
title = '%s Transit Timing Variation Analysis'%target
samples, results = TTVFIT.Fit(indir+'/results/3/',suffix,title)